## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle, pprint

%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((5*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:5].T.reshape(-1,2)

cv2.destroyAllWindows()
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.
objt =[]

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
   
    undist = cv2.undistort(img, mtx, dist, None, mtx)

    return undist, mtx, dist

row = 9

# Step through the list and search for chessboard corners
for index, fname in enumerate(images):
    cv2.destroyAllWindows()
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    if index==0:
        row=5
    else :
        row =6
    objp =[]
    objt=[]
    
    objp = np.zeros((row*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:row].T.reshape(-1,2)
    
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,row),None)
    objt = np.reshape(objp ,(1, objp.shape[0],objp.shape[1]))
    print(np.array(corners).shape)
    
    impoint = np.reshape(corners ,(1, corners.shape[0],corners.shape[1],corners.shape[2]))
    
    # If found, add object points, image points
    if ret == False:
        print("ouch : index " +str(index))
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img2 = cv2.drawChessboardCorners(img, (9,row), corners, ret)
        cv2.imshow('img',img2)
        cv2.waitKey(5000)

        print("objpoints shape : " + str(np.array(objpoints).shape))
        #print(objpoints)

        unimg, mtx, dist = cal_undistort(img, objt, impoint)
        cv2.imshow('unimg',unimg)
       

        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
        f.tight_layout()
        ax1.imshow(img2)
        ax1.set_title('Original Image', fontsize=50)
        ax2.imshow(unimg)
        ax2.set_title('Undistorted Image', fontsize=50)
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
        cv2.waitKey(500)
        print("Here : " + str(index))
        if index==0:
            plt.savefig('testplot.png')
            #save mtx & dist
            output = open('data.pkl', 'wb')
            # Pickle  using protocol 0.
            pickle.dump(mtx, output)
            pickle.dump(dist, output)
            print(mtx)
            print(dist)
            output.close()
            
        cv2.destroyAllWindows()
        
print("fin")


## And so on and so forth...